# 1. Setup
Install Confluent-Kafka client

In [25]:
pip install confluent-kafka


Note: you may need to restart the kernel to use updated packages.


In [27]:
pip show confluent-kafka

Name: confluent-kafka
Version: 2.3.0
Summary: Confluent's Python client for Apache Kafka
Home-page: https://github.com/confluentinc/confluent-kafka-python
Author: Confluent Inc
Author-email: support@confluent.io
License: 
Location: /Users/kellyliu/anaconda3/lib/python3.11/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# 2. Batch Processing

## 2.1. Load topic data from Confluent in batch mode

In [28]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [63]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0 pyspark-shell'


In [66]:
spark = SparkSession\

  .builder\

  .config("spark.jars", os.getcwd() + "/jars/spark-sql-kafka-0-10_2.12-3.2.1.jar" + "," + os.getcwd() + "/jars/kafka-clients-2.1.1.jar") \

  .appName("Structured_Redpanda_WordCount")\

  .getOrCreate()

spark.conf.set("spark.sql.shuffle.partitions", 1)

IndentationError: unexpected indent (1625169426.py, line 3)

In [65]:
# from pyspark.sql import SparkSession

# spark = SparkSession.builder \
#     .appName("KafkaIntegrationExample") \
#     .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
#     .getOrCreate()

from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("KafkaIntegration") \
    .set("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

topic_name = "topic_0"
bootstrap_servers = "pkc-4rn2p.canadacentral.azure.confluent.cloud:9092"
df_kafka = spark.read.format("kafka")\
    .option("kafka.bootstrap.servers", bootstrap_servers)\
    .option("subscribe", topic_name)\
    .option("kafka.security.protocol","SASL_SSL")\
    .option("kafka.sasl.mechanism", "PLAIN")\
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"DMEFD5LIKVBOJCWD\" password=\"mUEgMQvRZHnL4ZqtkTUww6Dywxxr9HoDmmgzYW6jRGumni3FrstD3c0ERxplHc8j\";")\
    .load()

display(df_kafka)

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.


## 2.2. Write a Kafka sink for batch queries

### 2.2.1. Create sample data

In [49]:
# import pyspark class Row from module sql
from pyspark.sql import *

In [50]:
# Create the sample data
sample_data = Row("key", "value", "topic")
sample1 = sample_data('1', '{"name": "Jane Doe", "age": 29, "email": "jane.doe@gmail.com"}', "test-topic")
sample2 = sample_data('2', '{"name": "Fatih", "age": 24, "email": "f.nayebi@gmail.com"}', "test-topic")
sample3 = sample_data('3', '{"name": "John Doe", "age": 35, "email": "john.doe@gmail.com"}', "test-topic")

print(sample1)

Row(key='1', value='{"name": "Jane Doe", "age": 29, "email": "jane.doe@gmail.com"}', topic='test-topic')


### 2.2.2. Create a dataframe from sample data

In [51]:
df = spark.createDataFrame([sample1, sample2, sample3])

In [52]:
df.show()

+---+--------------------+----------+
|key|               value|     topic|
+---+--------------------+----------+
|  1|{"name": "Jane Do...|test-topic|
|  2|{"name": "Fatih",...|test-topic|
|  3|{"name": "John Do...|test-topic|
+---+--------------------+----------+



In [54]:
display(df)

DataFrame[key: string, value: string, topic: string]

### 2.2.3. Write data from a dataframe to a confluent kafka topic

In [56]:
# Write key-value data from a DataFrame to a specific Kafka topic specified in an option
ds = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
    .write.format("kafka")\
    .option("kafka.bootstrap.servers", bootstrap_servers)\
    .option("subscribe", topic_name)\
    .option("kafka.security.protocol","SASL_SSL")\
    .option("kafka.sasl.mechanism", "PLAIN")\
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"DMEFD5LIKVBOJCWD\" password=\"mUEgMQvRZHnL4ZqtkTUww6Dywxxr9HoDmmgzYW6jRGumni3FrstD3c0ERxplHc8j\";")\
    .option("topic", topic_name)\
    .save()

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.

# 3. Stream Processing

## 3.1. Read a stream from Kafka

In [18]:
df_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap.servers)\
    .option("subscribe", topic_name)\
    .option("kafka.security.protocol","SASL_SSL")\
    .option("kafka.sasl.mechanism", "PLAIN")\
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"Replace with your api key\" password=\"replace with your api secret\";")\
    .load() \

display(df_stream)

key,value,topic,partition,offset,timestamp,timestampType


## 3.2. Write a Kafka sink for streaming queries

In [20]:
# Write key-value data from a DataFrame to a specific Kafka topic specified in an option
ds = df_stream \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
    .option("kafka.bootstrap.servers", bootstrap.servers)\
    .option("subscribe", topic_name)\
    .option("kafka.security.protocol","SASL_SSL")\
    .option("kafka.sasl.mechanism", "PLAIN")\
    .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username=\"Replace with your api key\" password=\"replace with your api secret\";")\
  .option("topic", "databricks_test") \
  .option("checkpointLocation", "/dbfs/dir") \
  .start()